# Fine-tuning Embeddings for RAG on Specific Data

As we start our "fine-tuning" week, we'll start with the lowest hanging improvement one can do for RAG - which is:

Fine-tuning embeddings!

- 🤝 Breakout Room #1:
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-m`
  - Task 5: Evaluating our Retriever

- 🤝 Breakout Room #2:
  - Task 1: Vibe Checking Our LCEL RAG Chain
  - Task 2: Ragas Evaluation



#### Basic Overview of Fine-tuning Embeddings

In essence, what we want to do when we fine-tune our embedding models is very simple:

```
Move the embeddings for questions relating to a document
closer together with that document
```

We can think of fine-tuning our embedding models as follows:

1) We have some pair of text items that *should* be closer together
  - `Question`, `Document` pairs
  - EX: `Who drives the bus?`, `The bus was driven by Kyle, the Bus Driver`.

2) We use these pairs as labeled data to fine-tune our embedding model.

The process of training helps the model more accurately associate our questions with the correct documents.

#####❓ Question #1:

Describe the nuance between using Q&D pairs to train the embedding model vs. inter-document pairs/related sentences.

What caveats does this approach have? Are there any special considerations for what kind of Q's we should use?

---

**ANSWER:**

We are specifically relating *the questions* to *the documents*. This means that we are making our embedding model at the very specific task of relating potential questions to specific documents.

There are many caveats, but the main ones are:

- Your Q's should reflect the Q's of your users
- This kind of fine-tuning will (purposefully) "overfit" on your data; this is the desired result in this case.

More to add:

Q&A pairs: 
- Task-Specific Training: This approach is particularly effective for tasks where the goal is to match questions with relevant documents or answers, such as in search engines or QA systems.
- Contextual Relevance: Q&D pairs train the model to understand the relationship between a query and a document, which can help in improving the retrieval of relevant information based on the query.
- Relevance: Use a diverse set of questions that reflect the actual queries users might have.

For training Q&A:
- Complexity: Include questions of varying complexity to ensure the model can handle both simple and complex queries.
- Coverage: Ensure questions cover a wide range of topics and contexts to enhance the model's versatility.

Inter-document pairs/related sentences:
- Semantic Similarity: This approach helps the model learn semantic similarity and how different pieces of text are related, which is useful for tasks like paraphrase detection or document clustering.
- Generalization: Training with related sentences or documents can lead to more generalizable embeddings that capture broader semantic relationships rather than being too narrowly focused on specific queries.
- Data Requirements: Requires a substantial amount of high-quality, related pairs to be effective. Gathering such data can be challenging.
- Contextual Nuance: It might not capture the nuances of how different contexts affect query-document relationships.

For training Q&A:
- Relevance: Ensure that the related pairs are truly similar or related to avoid training the model on misleading or noisy data.
- Variety: Use a diverse set of related pairs to help the model generalize better.

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [3]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [4]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [5]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1

### Provide OpenAI API Key

In [6]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


## Task 2: Loading Data

We'll be using a recent document released by the EU 'laying down harmonised rules on artificial intelligence and amending Regulations'.

The data can be found [here](https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A32024R1689), though we will be using a HTML version which was collected into the AIM DataRepository.

First, we'll clone and then `cd` into the DataRepository.

In [1]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 90 (delta 24), reused 29 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (90/90), 70.26 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [8]:
#%cd DataRepository

/content/DataRepository


In [2]:
!mv DataRepository/eu_ai_act.html .

Next we're going to be using the `UnstructuredHTMLLoader` to load our HTML document into a LangChain document using the [Unstructured](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.html.UnstructuredHTMLLoader.html) library.

In [9]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

training_documents_loaded = UnstructuredHTMLLoader("eu_ai_act.html")

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

Next we can load/split these documents as follows.

In [11]:
training_documents = text_splitter.split_documents(training_documents_loaded.load())

In [12]:
len(training_documents)

1029

In [13]:
training_documents[0]

Document(metadata={'source': 'eu_ai_act.html'}, page_content='Official Journal of the European Union EN L series\n\n2024/1689 12.7.2024\n\nREGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\nof 13 June 2024\n\nlaying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)\n\n(Text with EEA relevance)\n\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Articles 16 and 114 thereof,\n\nHaving regard to the proposal from the European Commission,')

Next, we're going to associate each of our chunks with a unique identifier.

In [14]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [15]:
training_documents[0]

Document(metadata={'source': 'eu_ai_act.html', 'id': 'c77ffb8b-6100-4d47-904a-e6b94b02c634'}, page_content='Official Journal of the European Union EN L series\n\n2024/1689 12.7.2024\n\nREGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\nof 13 June 2024\n\nlaying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)\n\n(Text with EEA relevance)\n\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Articles 16 and 114 thereof,\n\nHaving regard to the proposal from the European Commission,')

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [16]:
training_split_documents = training_documents[:300]
val_split_documents = training_documents[300:350]
test_split_documents = training_documents[350:400]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [July 18th](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [17]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [18]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [19]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

##### 🏗️ Activity #1:

We have:

- Lists of `Documents` with the `metadata` field `id`.

We need:

- An object with key `id`, which have values `str` questions.
- An object with key `question_id`, which have values `List(str)` which will be a list of associated `context_id`.

An Example:

question_object:
```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': 'What types of accessible formats are available for persons with disabilities?',
'df58ee4f-714c-419e-8324-94e5870574e2': 'How do accessible formats benefit persons with disabilities?',
'505fce8b-0e56-48de-a251-61027e396918': 'What are some of the risks associated with the increasing capabilities of AI systems that generate synthetic content?',
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': 'Why is it important for providers of AI systems to embed technical solutions for marking and detecting synthetic content?'
}
 ```

 context_object:
 ```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'df58ee4f-714c-419e-8324-94e5870574e2': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'505fce8b-0e56-48de-a251-61027e396918': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
}
 ```

 As you can see, a piece of context can be associated with more than 1 question.

 The task is to write the Python function(s) to accomplish this task.

 Your function signature is provided below, along with the desired return values.

 > NOTE: You can make any modifications that you desire - assuming that you have the correct input and outputs.

In [20]:
import tqdm
def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}
  for document in tqdm.tqdm(documents):
    #document_content = {"context" : document.page_content, "questions" : []}
    questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
    for question in questions_generated.content.split("\n"):
      question_id = str(uuid.uuid4())
      questions[question_id] = "".join(question.split(".")[1:]).strip()
      relevant_docs[question_id] = [document.metadata["id"]]
  return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

In [23]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, 2)

100%|██████████| 300/300 [04:37<00:00,  1.08it/s]


In [25]:
val_questions, val_relevant_contexts = create_questions(val_split_documents , 2)

100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


In [26]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 2)

100%|██████████| 50/50 [00:47<00:00,  1.06it/s]


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

> NOTE: If you ran into issues creating the data - you can use the data from the DataRespository. It's simply called: `train_dataset.jsonl`, etc.

In [27]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [28]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [29]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [49]:
!pip install -qU sentence_transformers datasets pyarrow==14.0.1

In [22]:
import pyarrow
pyarrow.__version__

'14.0.1'

In [21]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [30]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

> NOTE: It is typical to use a much larger batch size (~64+), hardware permitting.

In [31]:
BATCH_SIZE = 20

Let's move our dataset into the expected format for training.

In [32]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [33]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

Loss is an important part of training, fine-tuning, and more. If you want a deep dive on loss - you can check out our [event on loss!](https://www.youtube.com/watch?v=iB8FWR9aD5Q&t=8s).

The core loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

This is "wrapped" in `MatryoshkaLoss`, which you can read the implementation of [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [34]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

##### 🏗️ Activity #2:

Both of these losses sound "cool", but what are they - exactly - under the hood?

Why are these losses specifically doing? Please write a short summary of each loss.

> NOTE: This is a course focused on AI Engineering and the application of AI - looking for a hint? Try pasting the code (linked above) into ChatGPT/Claude to write the summary!

MultipleNegativesRankingLoss
- Purpose: MultipleNegativesRankingLoss is designed for training embeddings in scenarios where we have positive pairs of sentences (e.g., a query and a relevant document) and need to distinguish these from multiple negative samples (irrelevant documents).
- How It Works: 
    - This loss function operates on a batch of sentence pairs. Each anchor sentence in the batch is paired with one positive sentence and is used to compare against all other sentences in the batch as negatives.
    - It calculates similarity scores between the anchor and all other sentences, then applies a softmax function to these scores. The goal is to maximize the similarity between the anchor and the positive sentence while minimizing it with all other sentences (negatives).
    - The loss is computed using cross-entropy, which penalizes the model if the positive sentence's similarity score is not higher than that of the negative sentences.

MatryoshkaLoss
- Purpose: MatryoshkaLoss extends the idea of using multiple loss functions at different embedding dimensions. It allows us to apply a base loss function (like MultipleNegativesRankingLoss) across a series of progressively smaller embedding dimensions. This is useful for when training a model where users have the option to lower the embedding dimension to improve their embedding comparison speed and costs.

- How It Works:
    - The core idea is to train embeddings at different dimensions (e.g., 768, 512, 256, etc.), which can be useful for models that may need to operate at various dimensionalities depending on the deployment scenario.
    - It uses a specified base loss function and applies it at each of these dimensions. The loss values at each dimension are weighted according to specified weights and summed to get the final loss. In the code, we didn't input the weights, so the weights are set to 1 for all dimension
    - The loss function allows us to use smaller dimensions to speed up inference and reduce costs while still maintaining good performance during training.

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [35]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [36]:
EPOCHS = 5

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [37]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
30,No log,No log,0.870000,0.970000,0.990000,1.000000,0.870000,0.323333,0.198000,0.100000,0.870000,0.970000,0.990000,1.000000,0.940933,0.921250,0.921250,0.870000,0.970000,0.990000,1.000000,0.870000,0.323333,0.198000,0.100000,0.870000,0.970000,0.990000,1.000000,0.940933,0.921250,0.921250
50,No log,No log,0.880000,0.980000,0.990000,1.000000,0.880000,0.326667,0.198000,0.100000,0.880000,0.980000,0.990000,1.000000,0.945725,0.927500,0.927500,0.880000,0.980000,0.990000,1.000000,0.880000,0.326667,0.198000,0.100000,0.880000,0.980000,0.990000,1.000000,0.945725,0.927500,0.927500
60,No log,No log,0.880000,0.980000,0.990000,1.000000,0.880000,0.326667,0.198000,0.100000,0.880000,0.980000,0.990000,1.000000,0.946596,0.928667,0.928667,0.880000,0.980000,0.990000,1.000000,0.880000,0.326667,0.198000,0.100000,0.880000,0.980000,0.990000,1.000000,0.946596,0.928667,0.928667
90,No log,No log,0.880000,0.980000,1.000000,1.000000,0.880000,0.326667,0.200000,0.100000,0.880000,0.980000,1.000000,1.000000,0.948211,0.930667,0.930667,0.880000,0.980000,1.000000,1.000000,0.880000,0.326667,0.200000,0.100000,0.880000,0.980000,1.000000,1.000000,0.948211,0.930667,0.930667
100,No log,No log,0.880000,0.980000,1.000000,1.000000,0.880000,0.326667,0.200000,0.100000,0.880000,0.980000,1.000000,1.000000,0.948211,0.930667,0.930667,0.880000,0.980000,1.000000,1.000000,0.880000,0.326667,0.200000,0.100000,0.880000,0.980000,1.000000,1.000000,0.948211,0.930667,0.930667
120,No log,No log,0.870000,0.980000,1.000000,1.000000,0.870000,0.326667,0.200000,0.100000,0.870000,0.980000,1.000000,1.000000,0.944521,0.925667,0.925667,0.870000,0.980000,1.000000,1.000000,0.870000,0.326667,0.200000,0.100000,0.870000,0.980000,1.000000,1.000000,0.944521,0.925667,0.925667
150,No log,No log,0.870000,0.980000,0.990000,1.000000,0.870000,0.326667,0.198000,0.100000,0.870000,0.980000,0.990000,1.000000,0.944214,0.925333,0.925333,0.870000,0.980000,0.990000,1.000000,0.870000,0.326667,0.198000,0.100000,0.870000,0.980000,0.990000,1.000000,0.944214,0.925333,0.925333


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [38]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [46]:
test_dataset.keys()

dict_keys(['questions', 'relevant_contexts', 'corpus'])

In [50]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [51]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:33<00:00,  2.96it/s]


In [52]:
te3_results_df = pd.DataFrame(te3_results)

In [53]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

0.98

### `Snowflake/snowflake-arctic-embed-m` (base)

In [54]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 100/100 [00:01<00:00, 82.39it/s]


In [55]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [56]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.6

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [57]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:01<00:00, 87.52it/s]


In [58]:
finetune_results_df = pd.DataFrame(finetune_results)

In [59]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

1.0

# 🤝 Breakout Room #2

## Task 1: Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [60]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [61]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [62]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [63]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [64]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [65]:
base_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to promote a human-centric approach to AI, ensure the development of secure, trustworthy, and ethical AI, protect ethical principles, and facilitate the protection of natural persons, democracy, the rule of law, and environmental protection. Additionally, the regulation aims to boost innovation and employment, positioning the Union as a leader in the uptake of trustworthy AI.'

In [66]:
base_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'I do not know.'

In [67]:
base_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and competence in performing a wide range of tasks. Therefore, it can be inferred that having a billion parameters is a threshold for being considered a model with "too many" parameters in this context.'

In [68]:
base_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is a type of artificial intelligence (AI) technology designed to identify and interpret human emotions based on various inputs, such as facial expressions, voice tone, body language, or biometric data. These systems analyze patterns in the data to infer the emotional state of an individual.\n\nThe importance of emotion recognition systems lies in their potential applications across various fields, including mental health, customer service, security, and human-computer interaction. They can enhance user experiences, improve communication, and provide insights into emotional well-being. However, there are significant concerns regarding their reliability, specificity, and potential for discriminatory outcomes, particularly given the variability of emotional expression across different cultures and individuals.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [69]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [70]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [71]:
finetune_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to improve the functioning of the internal market, promote the uptake of human-centric and trustworthy artificial intelligence, ensure a high level of protection of health, safety, and fundamental rights, protect against the harmful effects of AI systems, and support innovation. The regulation aims to create a uniform legal framework that prevents Member States from imposing restrictions on the development, marketing, and use of AI systems, unless explicitly authorized by the regulation.'

In [72]:
finetune_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'Codes of practice are guidelines developed to ensure compliance with obligations for providers of general-purpose AI models, particularly those that present systemic risks. They aim to establish a risk taxonomy for systemic risks at the Union level, including their sources, and focus on specific risk assessment and mitigation measures. The AI Office is responsible for encouraging and facilitating the creation, review, and adaptation of these codes, which should reflect the state of the art and consider diverse perspectives. The codes of practice are intended to be a central tool for demonstrating compliance with relevant regulations and are expected to be ready by May 2, 2025.'

In [73]:
finetune_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and to competently perform a wide range of distinctive tasks. Therefore, it can be inferred that having at least a billion parameters is seen as a threshold for significant capability, but there is no specific limit stated for "too many" parameters.'

In [74]:
finetune_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is defined as an AI system that identifies or infers the emotions or intentions of natural persons based on their biometric data. This includes recognizing emotions such as happiness, sadness, anger, surprise, disgust, embarrassment, excitement, shame, contempt, satisfaction, and amusement. \n\nThe importance of emotion recognition systems lies in their potential applications across various fields, such as enhancing user experience in technology, improving mental health assessments, and facilitating better communication in human-computer interactions. However, there are significant concerns regarding their reliability, cultural variability in emotional expression, and the potential for discriminatory outcomes, which raises ethical considerations about their use.'

#####❓Question #2:

Which LCEL RAG Chain do you think answered the questions better, and why?

The fine-tuned one answer better cause:
1) it answered the questions that the basic rag didn't know
2) the answers are more accurate and easier to understand

## Task 2: RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [75]:
!pip install -qU ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.2/185.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.7 MB/s eta 0:00:00


### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [76]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

In [77]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
#I deleted the cell with run results cause the output is too long and crashed the notebook
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [79]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the role of the Commission in the supe...,[(162) To make best use of the centralised Uni...,The Commission is responsible for the supervis...,simple,"[{'source': 'eu_ai_act.html', 'id': '8ce2cb3e-...",True
1,What can downstream providers do if they suspe...,"[the AI Office, it should provide for the poss...",Downstream providers can lodge complaints if t...,simple,"[{'source': 'eu_ai_act.html', 'id': '469e6ae9-...",True
2,What is the purpose of joint activities propos...,[(160) The market surveillance authorities and...,The purpose of joint activities proposed by ma...,simple,"[{'source': 'eu_ai_act.html', 'id': '642331f0-...",True
3,What types of financial institutions are subje...,[undertakings and insurance holding companies ...,Insurance holding companies under Directive 20...,simple,"[{'source': 'eu_ai_act.html', 'id': '5441077b-...",True
4,What is the role of the European Data Protecti...,[take measures in relation to all AI systems w...,The European Data Protection Supervisor is des...,simple,"[{'source': 'eu_ai_act.html', 'id': 'd866672b-...",True


In [81]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the role of the Commission in the supe...,[(162) To make best use of the centralised Uni...,The Commission is responsible for the supervis...,simple,"[{'source': 'eu_ai_act.html', 'id': '8ce2cb3e-...",True
1,What can downstream providers do if they suspe...,"[the AI Office, it should provide for the poss...",Downstream providers can lodge complaints if t...,simple,"[{'source': 'eu_ai_act.html', 'id': '469e6ae9-...",True
2,What is the purpose of joint activities propos...,[(160) The market surveillance authorities and...,The purpose of joint activities proposed by ma...,simple,"[{'source': 'eu_ai_act.html', 'id': '642331f0-...",True
3,What types of financial institutions are subje...,[undertakings and insurance holding companies ...,Insurance holding companies under Directive 20...,simple,"[{'source': 'eu_ai_act.html', 'id': '5441077b-...",True
4,What is the role of the European Data Protecti...,[take measures in relation to all AI systems w...,The European Data Protection Supervisor is des...,simple,"[{'source': 'eu_ai_act.html', 'id': 'd866672b-...",True
5,How should national authorities report informa...,"[as appropriate, into their existing superviso...",National authorities should report any informa...,simple,"[{'source': 'eu_ai_act.html', 'id': 'd4c1360f-...",True
6,How should market surveillance authorities coo...,"[one purpose that is classified as high-risk, ...",Market surveillance authorities should coopera...,simple,"[{'source': 'eu_ai_act.html', 'id': 'ef0986e9-...",True
7,How often will the Commission evaluate and rev...,[(174) Given the rapid technological developme...,The Commission should evaluate and review this...,simple,"[{'source': 'eu_ai_act.html', 'id': '309175f8-...",True
8,What is the purpose of having a post-market mo...,[(155) In order to ensure that providers of hi...,The purpose of having a post-market monitoring...,simple,"[{'source': 'eu_ai_act.html', 'id': '1b686d82-...",True
9,How does the European Parliament participate i...,"[the preparation of delegated acts, the Europe...",The European Parliament participates in the pr...,simple,"[{'source': 'eu_ai_act.html', 'id': '3e9e18bb-...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [84]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm.tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [85]:
base_dataset = generate_answers(base_rag_chain, testset)

100%|██████████| 18/18 [00:17<00:00,  1.02it/s]


In [86]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 18/18 [00:23<00:00,  1.29s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [87]:
from ragas.metrics import (
    context_recall,
    context_precision,
)

In [88]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [89]:
result

{'context_precision': 0.4891, 'context_recall': 0.5463}

In [90]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What is the role of the Commission in the supe...,[3. Providers of general-purpose AI models sha...,The role of the Commission in the supervision ...,The Commission is responsible for the supervis...,0.816667,0.666667
1,What can downstream providers do if they suspe...,"[the results of its monitoring activities, or ...",Downstream providers can lodge complaints abou...,Downstream providers can lodge complaints if t...,1.000000,1.000000
2,What is the purpose of joint activities propos...,[of the high-risk AI system with the requireme...,I do not know.,The purpose of joint activities proposed by ma...,0.000000,0.000000
3,What types of financial institutions are subje...,[For deployers that are financial institutions...,The context does not specify particular types ...,Insurance holding companies under Directive 20...,1.000000,1.000000
4,What is the role of the European Data Protecti...,[and without bias. Although the majority of AI...,The European Data Protection Supervisor acts a...,The European Data Protection Supervisor is des...,1.000000,0.500000


In [91]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [92]:
result

{'context_precision': 0.8206, 'context_recall': 0.8843}

In [93]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What is the role of the Commission in the supe...,"[SECTION 5\n\nSupervision, investigation, enfo...",The Commission has exclusive powers to supervi...,The Commission is responsible for the supervis...,1.000000,1.0
1,What can downstream providers do if they suspe...,[2. Downstream providers shall have the right ...,Downstream providers can lodge a complaint all...,Downstream providers can lodge complaints if t...,0.876667,1.0
2,What is the purpose of joint activities propos...,[11. Market surveillance authorities and the C...,The purpose of joint activities proposed by ma...,The purpose of joint activities proposed by ma...,1.000000,0.5
3,What types of financial institutions are subje...,[4. For providers that are financial instituti...,The types of financial institutions subject to...,Insurance holding companies under Directive 20...,1.000000,1.0
4,What is the role of the European Data Protecti...,"[9. Where Union institutions, bodies, offices ...",The European Data Protection Supervisor acts a...,The European Data Protection Supervisor is des...,1.000000,0.5


#### 🏗️ Activity #3:

Discuss changes that you'd make to this pipeline based on the performance improvements that you see with RAGAS and the fine-tuning.

Come up with 3 changes, and then we'll discuss these options as a group!

1. query expansion techniques. For example find ways to enhancing the original query with additional terms or synonyms to capture a broader range of relevant documents.
2. check further fine-tuning or domain adaptation of embeddings could yield additional improvements, For example, add customized tokenizer, text_spliter, etc. 
3. apply re-ranking techniques after retriever, add metadata filtering which is able to narrow down the retrieved documents based on additional information associated with each document, such as document type, source, date, etc. 
4. trying different chunk_size, llm